### IMPORTS

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, roc_curve
from sklearn.svm import SVR

### LOAD DATAFRAME

In [3]:
pd.options.display.max_columns
df = pd.read_csv('df_all_rs.csv')
df = df.drop('Unnamed: 0', axis=1)
df = df.drop('index', axis=1)
df = df.sort_values(by="GAME_DATE_EST")
df

,GAME_DATE_EST,GAME_ID,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,FG3_PCT_home,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
1188,2003-10-28,20300003,1610612747,1610612742,2003,1610612747,109.0,0.506,0.600,0.350,32.0,46.0,1610612742,93.0,0.376,0.733,0.280,17.0,46.0,1
1187,2003-10-28,20300002,1610612759,1610612756,2003,1610612759,83.0,0.425,0.769,0.100,20.0,38.0,1610612756,82.0,0.361,0.810,0.500,14.0,43.0,1
1186,2003-10-28,20300001,1610612755,1610612748,2003,1610612755,89.0,0.440,0.533,0.350,25.0,39.0,1610612748,74.0,0.408,0.824,0.250,16.0,44.0,1
1185,2003-10-29,20300012,1610612762,1610612757,2003,1610612762,99.0,0.575,0.714,0.556,25.0,29.0,1610612757,92.0,0.429,0.762,0.333,20.0,40.0,1
1184,2003-10-29,20300006,1610612740,1610612737,2003,1610612740,88.0,0.324,0.700,0.160,24.0,55.0,1610612737,83.0,0.398,0.737,0.214,18.0,58.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18173,2019-04-10,21801225,1610612763,1610612744,2018,1610612763,132.0,0.490,0.833,0.457,25.0,49.0,1610612744,117.0,0.500,0.857,0.433,32.0,39.0,1
18172,2019-04-10,21801224,1610612755,1610612741,2018,1610612755,125.0,0.559,0.600,0.462,30.0,47.0,1610612741,109.0,0.474,0.667,0.375,21.0,39.0,1
18171,2019-04-10,21801223,1610612752,1610612765,2018,1610612752,89.0,0.403,0.720,0.300,23.0,33.0,1610612765,115.0,0.482,0.950,0.424,25.0,56.0,0
18170,2019-04-10,21801222,1610612766,1610612753,2018,1610612766,114.0,0.526,0.889,0.276,21.0,32.0,1610612753,122.0,0.545,0.750,0.367,24.0,39.0,0


### EVALUATION FUNCTION

In [4]:
def evaluar_metricas(estimator, data, target, name):
    
    X_train, X_test, y_train, y_test = train_test_split(data,target)

    estimator.fit(X_train, y_train)
    y_hat = estimator.predict(X_test)

    errors = y_test - y_hat
    mae = np.mean(np.abs(errors))
    mse = np.mean(errors ** 2)
    rmse = np.sqrt(mse)
    
    return [name, mse, rmse, mae]

### TARGET AND DATA SELECTION

#### 1) FG_PCT_home, FG_PCT_away, FG3_PCT_home, FG3_PCT_away

In [5]:
target_1 = df['HOME_TEAM_WINS']
data_1 = df[['FG_PCT_home', 'FG_PCT_away', 'FG3_PCT_home', 'FG3_PCT_away']]

data_copy_1 = data_1.copy()
normalized_data_1 = (data_copy_1 - data_copy_1.mean()) / data_copy_1.std()

#### 2) FG_PCT_home, FG_PCT_away, FG3_PCT_home, FG3_PCT_away, AST_home, REB_home, AST_away, REB_away

In [6]:
target_2 = df['HOME_TEAM_WINS']
data_2 = df[['FG_PCT_home','FG_PCT_away','FG3_PCT_home','FG3_PCT_away','AST_home','REB_home','AST_away','REB_away']]

data_copy_2 = data_2.copy()
normalized_data_2 = (data_copy_2 - data_copy_2.mean()) / data_copy_2.std()

#### ¿QUE OPCION ES MEJOR?, COMO DETERMINO SI DEBO COGER MAS O MENOS ESTADISTICAS PARA EL ALGORITMO

### ALGORITHMS:

### K Neighbors Regressor (KNR)

In [6]:
metrics_results_knr_1 = pd.DataFrame(columns=['NAME', 'MSE','RMSE', 'MAE'])

In [7]:
for n in range(1, 21):    
    for i in range(1, 501, 20):
        knr = KNeighborsRegressor(n_neighbors=i)
        metrics_results_knr_1.loc[len(metrics_results_knr_1)+1] = \
            evaluar_metricas(knr, 
                             normalized_data_1,
                             target_1, 
                             'knr_normalized_neighbors_'+str(i))

metrics_results_knr_1.to_csv('metrics_results_knr_1.csv')

In [8]:
metrics_results_knr_2 = pd.DataFrame(columns=['NAME', 'MSE','RMSE', 'MAE'])

In [9]:
for n in range(1, 21):    
    for i in range(1, 501, 20):
        knr = KNeighborsRegressor(n_neighbors=i)
        metrics_results_knr_2.loc[len(metrics_results_knr_2)+1] = \
            evaluar_metricas(knr, 
                             normalized_data_2,
                             target_2, 
                             'knr_normalized_neighbors_'+str(i))

metrics_results_knr_2.to_csv('metrics_results_knr_2.csv')

### Decision Tree Regressor (DTR)

In [11]:
metrics_results_dtr_1 = pd.DataFrame(columns=['NAME', 'MSE','RMSE', 'MAE'])

In [12]:
for n in range(1, 21):  
    for i in range(1, 301, 30):
        for j in range(1, 301, 30):
            dtr = DecisionTreeRegressor(max_depth=i, min_samples_leaf=j)
            metrics_results_dtr_1.loc[len(metrics_results_dtr_1)+1] = \
                evaluar_metricas(dtr, 
                                 normalized_data_1, 
                                 target_1, 
                                 'dtr_normalized__maxdepth_'+str(i)+'__minsamplesleaf_'+str(j))
            
metrics_results_dtr_1.to_csv('metrics_results_dtr_1.csv')

In [13]:
metrics_results_dtr_2 = pd.DataFrame(columns=['NAME', 'MSE','RMSE', 'MAE'])

In [14]:
for n in range(1, 21):  
    for i in range(1, 301, 30):
        for j in range(1, 301, 30):
            dtr = DecisionTreeRegressor(max_depth=i, min_samples_leaf=j)
            metrics_results_dtr_2.loc[len(metrics_results_dtr_2)+1] = \
                evaluar_metricas(dtr, 
                                 normalized_data_2, 
                                 target_2, 
                                 'dtr_normalized__maxdepth_'+str(i)+'__minsamplesleaf_'+str(j))
            
metrics_results_dtr_2.to_csv('metrics_results_dtr_2.csv')

### Random Forest Regressor (RFR)

In [15]:
metrics_results_rfr_1 = pd.DataFrame(columns=['NAME', 'MSE','RMSE', 'MAE'])

In [16]:
for n in range(1, 21): 
    for k in range(1, 101, 10):
        for i in range(1, 101, 10):
            for j in range(1, 101, 10):
                rfr = RandomForestRegressor(n_estimators=k, max_depth=i, min_samples_leaf=j)
                metrics_results_rfr_1.loc[len(metrics_results_rfr_1)+1] = \
                    evaluar_metricas(rfr, 
                                     normalized_data_1,
                                     target_1,
                                     'rfr_normalized__estimators_'+str(k)+
                                     '__maxdepth_'+str(i)+
                                     '__minsamplesleaf_'+str(j))
    
metrics_results_rfr_1.to_csv('metrics_results_rfr_1.csv')

In [11]:
metrics_results_rfr_2 = pd.DataFrame(columns=['NAME', 'MSE','RMSE', 'MAE'])

In [12]:
for n in range(1, 21): 
    for k in range(1, 101, 10):
        for i in range(1, 101, 10):
            for j in range(1, 101, 10):
                rfr = RandomForestRegressor(n_estimators=k, max_depth=i, min_samples_leaf=j)
                metrics_results_rfr_2.loc[len(metrics_results_rfr_2)+1] = \
                    evaluar_metricas(rfr, 
                                     normalized_data_2,
                                     target_2,
                                     'rfr_normalized__estimators_'+str(k)+
                                     '__maxdepth_'+str(i)+
                                     '__minsamplesleaf_'+str(j))
    
metrics_results_rfr_2.to_csv('metrics_results_rfr_2.csv')

### Multi-layer Perceptron Regressor (MLPR)

In [ ]:
metrics_results_mlpr_1 = pd.DataFrame(columns=['NAME', 'MSE','RMSE', 'MAE'])

In [ ]:
activations = ['relu', 'logistic', 'tanh']
solvers = ['adam', 'sgd']
learning_rates = ['constant', 'invscaling', 'adaptive']
batch_sizes = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]

for n in range(1, 21): 
    for act in activations:
        for sol in solvers:
            for rate in learning_rates:
                for size in batch_sizes:
                    mlpr = MLPRegressor(activation=act, solver=sol, batch_size=size, learning_rate=rate)
                    metrics_results_mlpr_1.loc[len(metrics_results_mlpr_1)+1] = \
                        evaluar_metricas(mlpr, 
                                         normalized_data_1,
                                         target_1, 
                                         'mlpr_normalized__activation_'+act+
                                         '__solver_'+sol+
                                         '__batchsize_'+str(size)+
                                         '__learningrate_'+rate)
                
metrics_results_mlpr_1.to_csv('metrics_results_mlpr_1.csv')

In [ ]:
metrics_results_mlpr_2 = pd.DataFrame(columns=['NAME', 'MSE','RMSE', 'MAE'])

In [ ]:
activations = ['relu', 'logistic', 'tanh']
solvers = ['adam', 'sgd']
learning_rates = ['constant', 'invscaling', 'adaptive']
batch_sizes = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500]

for n in range(1, 21): 
    for act in activations:
        for sol in solvers:
            for rate in learning_rates:
                for size in batch_sizes:
                    mlpr = MLPRegressor(activation=act, solver=sol, batch_size=size, learning_rate=rate)
                    metrics_results_mlpr_2.loc[len(metrics_results_mlpr_2)+1] = \
                        evaluar_metricas(mlpr, 
                                         normalized_data_2,
                                         target_2, 
                                         'mlpr_normalized__activation_'+act+
                                         '__solver_'+sol+
                                         '__batchsize_'+str(size)+
                                         '__learningrate_'+rate)
                
metrics_results_mlpr_2.to_csv('metrics_results_mlpr_2.csv')

### Logistic Regressor (LogR)

In [ ]:
metrics_results_logr_1 = pd.DataFrame(columns=['NAME', 'MSE','RMSE', 'MAE'])

In [ ]:
for n in range(1, 21): 
    for i in range(1,51):
        logR = LogisticRegression()
        metrics_results_logR_1.loc[len(metrics_results_logR_1)+1] = \
            evaluar_metricas(logR, 
                             normalized_data_1, 
                             target_1, 
                             'logR_normalized_'+str(i))
        
metrics_results_logR_1.to_csv('metrics_results_logR_1.csv')

In [ ]:
metrics_results_logr_2 = pd.DataFrame(columns=['NAME', 'MSE','RMSE', 'MAE'])

In [ ]:
for n in range(1, 21): 
    for i in range(1,51):
        logR = LogisticRegression()
        metrics_results_logR_2.loc[len(metrics_results_logR_2)+1] = \
            evaluar_metricas(logR, 
                             normalized_data_2, 
                             target_2, 
                             'logR_normalized_'+str(i))
        
metrics_results_logR_2.to_csv('metrics_results_logR_2.csv')

### DATAFRAME CREATION FOR ALL METRICS OF DIFFERENTS ALGORITHMS

In [ ]:
df_metrics_list_1 = [metrics_results_knr_1, metrics_results_dtr_1, metrics_results_rfr_1, metrics_results_mlpr_1,
                     metrics_results_logr_1]

df_all_metrics_1 = pd.concat(df_metrics_list_1)

df_all_metrics_1.to_csv('metrics_results_compilation_1.csv')

In [10]:
df_metrics_list_2 = [metrics_results_knr_2, metrics_results_dtr_2, metrics_results_rfr_2, metrics_results_mlpr_2,
                     metrics_results_logr_2]

df_all_metrics_2 = pd.concat(df_metrics_list_2)

df_all_metrics_2.to_csv('metrics_results_compilation_2.csv')

NameError: name 'metrics_results_dtr_2' is not defined

### OPTIMIZACION DE HYPERPARAMETROS CON: 

### Hiperparámetros predeterminados, GridSearchCV, RandomizedSearchCV, Hyperopt

In [24]:
#Vamos a crear un dataframe donde recojamos los resultado de aplicar las distintas tecnicas de optimizacion
#sobre los algoritmos que hemos seleccionado

In [7]:
x = df[['FG_PCT_home','FG_PCT_away','FG3_PCT_home','FG3_PCT_away','AST_home','REB_home','AST_away','REB_away']]
y = df['HOME_TEAM_WINS']

X = x.copy()
X_normalized = (X - X.mean()) / X.std()

In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(X_normalized,y,train_size=0.70,test_size=0.30,random_state=0)

In [9]:
#Dataframe de resultados de accuracy
columnas = ['Tecnica','KNR','DTR','RFR','MLPR','LogR','SVR']

df_opt = pd.DataFrame(columns=columnas)
df_opt.set_index("Tecnica",inplace=True)
df_opt.loc['Standard'] = [0,0,0,0,0,0]
df_opt.loc['GridSearch'] = [0,0,0,0,0,0]
df_opt.loc['RandomSearch'] = [0,0,0,0,0,0]
df_opt.loc['Hyperopt'] = [0,0,0,0,0,0]
df_opt.head()

,KNR,DTR,RFR,MLPR,LogR,SVR
Tecnica,,,,,,
Standard,0,0,0,0,0,0
GridSearch,0,0,0,0,0,0
RandomSearch,0,0,0,0,0,0
Hyperopt,0,0,0,0,0,0


### 1) Hiperparámetros predeterminados

In [10]:
knr = KNeighborsRegressor()
dtr = DecisionTreeRegressor()
rfr = RandomForestRegressor()
mlpr = MLPRegressor()
logR = LogisticRegression()
svr = SVR()


models = [knr,dtr,rfr,mlpr,logR,svr]

col = 0
for model in models:
    model.fit(X_train,y_train.values.ravel())
    df_opt.iloc[0,col] = model.score(X_valid,y_valid)
    col += 1

df_opt.head()

,KNR,DTR,RFR,MLPR,LogR,SVR
Tecnica,,,,,,
Standard,0.428437,-0.036148,0.495903,0.522937,0.837457,0.498843
GridSearch,0,0,0,0,0,0
RandomSearch,0,0,0,0,0,0
Hyperopt,0,0,0,0,0,0


### 2) GridSearchCV

In [11]:
# KNeighborsRegressor
n_neighbors = range(1, 11, 2)
weights = ['uniform', 'distance']
#algorithm = ['auto', 'ball_tree', 'kd_tree', 'brute']
#leaf_size = range(1,41,2)
#metric = ['euclidean', 'manhattan', 'minkowski']

knr_grid = dict(n_neighbors=n_neighbors, 
                weights=weights)


# DecisionTreeRegressor
#criterion = ['squared_error', 'friedman_mse', 'absolute_error', 'poisson']
max_depth = range(1, 11, 2)
min_samples_leaf = range(1, 11, 2)

dtr_grid = dict(max_depth=max_depth,
                min_samples_leaf=min_samples_leaf)


# RandomForestRegressor
n_estimators = [10, 100, 1000]
max_depth = range(1, 11, 2)
min_samples_leaf = range(1, 11, 2)
max_features = ['sqrt', 'log2']

rfr_grid = dict(n_estimators=n_estimators, 
                max_depth=max_depth, 
                min_samples_leaf=min_samples_leaf,
                max_features=max_features)


# MLPRegressor
activation = ['relu', 'logistic', 'tanh']
solver = ['adam', 'sgd', 'lbfgs']
learning_rate = ['constant', 'invscaling', 'adaptive']
#batch_size = range(1, 501, 50)

mlpr_grid = dict(activation=activation,
                 solver=solver,
                 learning_rate=learning_rate)


# LogisticRegression
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]
#class_weight

logR_grid = dict(solver=solvers,
                 penalty=penalty,
                 C=c_values)


#SupportVectorRegressor
kernels =['linear', 'poly', 'rbf', 'sigmoid', 'precomputed']
c_values = [100, 10, 1.0, 0.1, 0.01]

svr_grid = dict(kernel=kernels,
                C=c_values)

In [12]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold

In [13]:
models = [knr,dtr,rfr,mlpr,logR,svr]

grids = [knr_grid, dtr_grid, rfr_grid, mlpr_grid, logR_grid, svr_grid]
col = 0

for ind in range(0,len(models)):
    cv = RepeatedStratifiedKFold(n_splits=10, 
                                 n_repeats=3, 
                                 random_state=1)
    
    grid_search = GridSearchCV(estimator=models[col],
                               param_grid=grids[col], 
                               n_jobs=-1, 
                               cv=cv,
                               scoring='accuracy',
                               error_score=0)
    
    grid_clf_acc = grid_search.fit(X_train, y_train)
    df_opt.iloc[1,col] = grid_clf_acc.score(X_valid,y_valid)
    col += 1

df_opt.head()

/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to 0. Details: 
Traceback (most recent call last):
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 93, 

/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to 0. Details: 
Traceback (most recent call last):
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 93, 

/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to 0. Details: 
Traceback (most recent call last):
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 93, 

/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to 0. Details: 
Traceback (most recent call last):
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 93, 

/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to 0. Details: 
Traceback (most recent call last):
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 93, 

...

/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to 0. Details: 
Traceback (most recent call last):
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 93, 

  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 93, in _check_targets
    raise ValueError(
ValueError: Classification metrics can't handle a mix of binary and continuous targets

  warnings.warn(
/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to 0. Details: 
Traceback (most recent call last):
  File "/home/dsc/a

ValueError: Classification metrics can't handle a mix of binary and continuous targets

/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to 0. Details: 
Traceback (most recent call last):
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 93, 

/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:770: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to 0. Details: 
Traceback (most recent call last):
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py", line 761, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 216, in __call__
    return self._score(
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_scorer.py", line 264, in _score
    return self._sign * self._score_func(y_true, y_pred, **self._kwargs)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 211, in accuracy_score
    y_type, y_true, y_pred = _check_targets(y_true, y_pred)
  File "/home/dsc/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py", line 93, 

### 3) RandomizedSearchCV

In [ ]:
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV

In [ ]:
col = 0
for ind in range(0,len(models)):
    cv = RepeatedStratifiedKFold(n_splits=10, 
                                 n_repeats=3,
                                 random_state=1)
    n_iter_search = 3
    random_search = RandomizedSearchCV(models[col],
                                       param_distributions=grids[col],
                                       n_iter=n_iter_search, 
                                       cv=cv)
    
    random_search.fit(X_train,y_train)
    df_opt.iloc[2,col] = random_search.score(X_valid,y_valid)
    col += 1
    
df_opt.head()